In [34]:
import os
import json
from web3 import Web3
from pathlib import Path
from dotenv import load_dotenv
import streamlit as st
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
from PIL import Image
load_dotenv()

True

In [35]:
contract_address = os.getenv("SMART_CONTRACT_ADDRESS")

In [36]:
print(contract_address)

0xeD858A763Df965B09d2f4a2354E45E6186BE10C7


In [37]:
w3 = Web3(Web3.HTTPProvider(os.getenv("WEB3_PROVIDER_URI")))

In [38]:

#@st.cache_resource()
def load_contract(w3):

    # Load the contract ABI
    with open(Path('./contracts/compiled/mem_token_abi.json')) as f:
        mem_token_abi = json.load(f)

    contract_address = os.getenv("SMART_CONTRACT_ADDRESS")

    # Load the contract
    contract = w3.eth.contract(
        address=contract_address,
        abi=mem_token_abi
    )

    return contract



In [39]:
contract = load_contract(w3)

In [40]:
print(contract)

In [50]:
all_accounts = w3.eth.accounts
all_accounts

['0x9990CA430403CC9A90b874BaF1aD877a56FA3a9b',
 '0xfe28b170E3F283DaA8A98ef84A09ff0A004F37F3',
 '0xcb36e342787D12157C85f3618B3d1839494167F3',
 '0x7b442a7044977e1439b74f23d9b77d6663cE3410',
 '0x0DeA5005b7585721dd83b82c5f9f61c772129392',
 '0xb9575455E6339A97D10D00E2957832cEF3DDBE59',
 '0x783bC8d62c25003b724F6EB3e6482D8BC36F32aC',
 '0x8039511cA043613A0C7a36505E888c0B1c5fa679',
 '0x08a72D2E1C574F76C61eBFf47c9290dB8bE37CF9',
 '0xFd86A5584BDf7eBfB0Be82a5fcD9B93bC92171c2']

In [58]:
from faker import Faker
fake = Faker()



# Connect to database
connection = psycopg2.connect(
    dbname="fintech_project_3",
    user="postgres",
    password="postgres",
    host="localhost",
    port=5432
)

cursor = connection.cursor()

traitIndex = 4

for i in range(47):
    
    first_name = fake.first_name()
    last_name = fake.last_name()
    street_address = fake.street_address()
    city = fake.city()
    state = fake.state()
    zip_code = fake.postalcode()
    for i in all_accounts:
        wall_bal = float(round(w3.fromWei(w3.eth.get_balance(i), "ether"),2))
        if wall_bal < 150:
            if wall_bal > 1:
                address = i
    
    # Call trait price in wei for selected traitIndex
    trait_price_wei = w3.toWei(contract.functions.getMembershipInfo(traitIndex).call()[1], 'wei')
    # Convert price to ETH
    trait_price_eth= w3.fromWei(trait_price_wei, 'ether') 
    
    # Use the contract to send a transaction to the mintMembership function
    tx_hash = contract.functions.mintMembership(
        address,
        traitIndex
    ).transact({'from': address, 'gas': 1000000, 'value': trait_price_wei})
    receipt = w3.eth.waitForTransactionReceipt(tx_hash)

    # Store other values for db insert
    tx_hash_value=receipt['transactionHash'].hex()
    traitIndex_value=traitIndex
    eth_price_value=trait_price_eth
    gas_cost_value= receipt['gasUsed']

    insert_query = """
    INSERT INTO customer_data (first_name, last_name, street_address, city, cust_state, zip_code, tx_hash, traitIndex, eth_price, gas_cost)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    data_to_insert = (first_name, last_name, street_address, city, state, zip_code, tx_hash_value, traitIndex_value, eth_price_value, gas_cost_value)

    cursor.execute(insert_query, data_to_insert)
    connection.commit()
    

cursor.close()
connection.close()